In [2]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [4]:
import os
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy.spatial.distance import cdist

from traffic.core import Traffic, Flight
from traffic.data import airports
from traffic.drawing import countries
import traffic.core.geodesy as geo

X_not_sampled = pd.read_pickle("Data/X_raw.pkl")
Real = pd.read_pickle("Data/distributions_along_lines.pkl")

# Generated
Vines_samp = pd.read_pickle("Data/generated_vines_and_sampling.pkl")
Mvn_samp = pd.read_pickle("Data/generated_MVN_and_sampling.pkl")
Gm_samp = pd.read_pickle("Data/generated_GM_and_sampling.pkl")
Mvn_Wsamp = pd.read_pickle("Data/generated_MVN_without_sampling.pkl")
Gm_Wsamp = pd.read_pickle("Data/generated_GM_without_sampling.pkl")
Vines_Wsamp = pd.read_pickle("Data/generated_vines_without_sampling.pkl")

# traffic resampled
t_resampled = Traffic.from_file("Data/GA_resampled.parquet")
features = ["x", "y"]
X_sampled = np.stack(list(f.data[features].values.ravel() for f in t_resampled))

# Normals for sampling
perpendiculars = pd.read_parquet("Data/Normals_sampling.parquet")
perpendiculars["angle"] = np.arctan(perpendiculars.m)

<IPython.core.display.Javascript object>

In [5]:
from scipy.spatial import distance


def energy_distance(x, y):
    n1 = x.shape[0]
    n2 = y.shape[0]
    a = cdist(x, y, "euclidean").mean()
    b = cdist(x, x, "euclidean").mean()
    c = cdist(y, y, "euclidean").mean()
    e = (n1 * n2 / (n1 + n2)) * (2 * a - b - c)
    return e

<IPython.core.display.Javascript object>

In [8]:
def inverse_sampling(df):
    n_gen = df.shape[0]
    n_feat = df.shape[1]
    res = np.zeros((n_gen, n_feat*2))
    res[:,0::2] = np.array(df) * np.array([np.cos(perpendiculars.angle.values),] * n_gen) + np.array(
    [perpendiculars.x.values,] * n_gen
    )
    res[:,1::2] = perpendiculars.m.values * res[:,0::2] + perpendiculars.p.values
    return res


<IPython.core.display.Javascript object>

In [26]:
from sklearn.preprocessing import StandardScaler

x = StandardScaler().fit_transform(Real)
y1 = StandardScaler().fit_transform(Gm_samp)
y2 = StandardScaler().fit_transform(Vines_samp)
y3 = StandardScaler().fit_transform(Mvn_samp)
print(energy_distance(x, y1))
print(energy_distance(x, y2))
print(energy_distance(x, y3))

3.376426585163
4.2951681744212475
17.472542935898275


<IPython.core.display.Javascript object>

Pour faire le test statistique :
- On calcule e0, la e-distance entre X et Y
- On choisit le nombre d'échantillonage du boostrap B, et les tailles des échantillons n_i, et  on pose ek = 0
- Pour chaque échantillonnage, on tire n_i observations dans la matrice concaténée [X,Y]
- On calcule e[b] pour léchantillonage, et si e[b] > e0, alors on incrémente ek
- La p-valeur vaut (ek+1)/(B+1) (probabilité de rejeter l'hypothèse nulle alors qu'elle est vraie, ie proba de se retrouver dans les queues de la distribution de H0, ie probabilité sous H0 d'observer une valeur encore plus extrême que celle observée)

Le dernier point est logique. On observe la statistique de test e0. On veut savoir si, sous l'hypothèse nulle, elle est considéreée comme improbable pour pas. Donc quelle est la probabilité d'obtenir une valeur encore plus grande que celle de e0. Cette probabilité est estimée par le bootstrap, en considérant que X et Y viennent de la même distribution. On calcule alors plusieurs fois e[b], et on compte le nombre de fois où on a effectivement une valeur plus grande que e0. On obtient alors une approximation de la probabilité d'obtenir une valeur plus grande que e0, si on considère que X et Y sont identiquement distribués.

La question se pose alors de l'estimation de cette probabilité par le bootstrap. Si X et Y ne sont pas ID, la probabilité estimée est-elle toujours vérifiée ? Voir the permutation test approach
outlined e.g. by Efron (1993, Chapter 15)

!!! Attention dans cette version on fait du bootstrap avec remise, contrairement au papier

La taille de chaque bootstrap est la taille totale du dataset pooled. Pendant l'étape de bootstrap l'échantillon formé 1 est de taille n1 et l'échantillon 2 de taille n2

In [69]:
dataset1 = X_sampled
dataset2 = inverse_sampling(Bm_samp)
B = 2000


def Etest(dataset1, dataset2, B):
    n1 = len(dataset1)
    n2 = len(dataset2)
    ek = 0
    e0 = energy_distance(dataset1, dataset2)
    pop = np.array(pd.concat([dataset1, dataset2], ignore_index=True))
    for i in tqdm(range(B)):
        np.random.shuffle(pop)
        eb = energy_distance(pop[:n1, :], pop[n1:, :])
        if eb > e0:
            ek += 1

    return e0, (ek + 1) / (B + 1)


Etest(pd.DataFrame(dataset1), pd.DataFrame(dataset2), B)

  0%|          | 0/2000 [00:00<?, ?it/s]

(9041.409159639383, 0.9435282358820589)

<IPython.core.display.Javascript object>

In [72]:
print(Etest(pd.DataFrame(X_sampled), pd.DataFrame(inverse_sampling(Gm_samp)), B))
print(Etest(pd.DataFrame(X_sampled), pd.DataFrame(inverse_sampling(Vines_samp)), B))
print(Etest(pd.DataFrame(X_sampled), pd.DataFrame(inverse_sampling(Mvn_samp)), B), "\n")

print(Etest(pd.DataFrame(X_not_sampled), pd.DataFrame(Gm_Wsamp), B))
print(Etest(pd.DataFrame(X_not_sampled), pd.DataFrame(Vines_Wsamp), B))
print(Etest(pd.DataFrame(X_not_sampled), pd.DataFrame(Mvn_Wsamp), B))

  0%|          | 0/2000 [00:00<?, ?it/s]

(5487.7183758251795, 1.0)


  0%|          | 0/2000 [00:00<?, ?it/s]

(9041.409159639383, 0.9465267366316842)


  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

(36565.28149000713, 0.18490754622688654)


  0%|          | 0/2000 [00:00<?, ?it/s]

(53731.40587115893, 0.04997501249375312)


<IPython.core.display.Javascript object>

In [28]:
print(energy_distance(X_sampled, inverse_sampling(Gm_samp)))
print(energy_distance(X_sampled, inverse_sampling(Vines_samp)))
print(energy_distance(X_sampled, inverse_sampling(Mvn_samp)), "\n")

print(energy_distance(X_not_sampled, Gm_Wsamp))
print(energy_distance(X_not_sampled, Vines_Wsamp))
print(energy_distance(X_not_sampled, Mvn_Wsamp))

5487.7183758251795
9041.409159639383
48076.605877948365 

9208.959841907183
36565.28149000713
53731.40587115893


<IPython.core.display.Javascript object>